# Deploying a simple RAG Application using an API

In [ ]:
!pip install graphlit-client -q

In [ ]:
from typing import Optional
from graphlit import Graphlit
from graphlit_api import *
import time

In [ ]:
# Initialize the Graphlit client using organization and environment details.
graphlit = Graphlit(
    organization_id="insertkey",
    environment_id="insertkey",
    jwt_secret="insertkey"
)

In [ ]:
# Function to create a new feed using the provided URL (uri) and return the feed ID or an error message.
async def create_feed(graphlit, uri):
    input = FeedInput(
        name=uri,  # The name of the feed is set to the URL itself.
        type=FeedTypes.WEB,  # Specifies that the feed type is a web feed.
        web=WebFeedPropertiesInput(
            uri=uri,  # URL for the web feed.
            readLimit=10  # Sets the limit on the number of reads allowed.
        )
    )

    try:
        # Attempt to create the feed using the graphlit client.
        response = await graphlit.client.create_feed(input)

        # Extract and return the feed ID from the response.
        feed_id = response.create_feed.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return feed_id

# Function to create a specification for a summarization task using various ANTHROPIC models.
async def create_specification(graphlit, model_version, temperature=0.1, probability=0.2, token_limit=2048):
    input = SpecificationInput(
        name=f"Summarization with {model_version}",  # Name of the specification, including model version.
        type=SpecificationTypes.COMPLETION,  # Specifies that this is a completion type specification.
        serviceType=ModelServiceTypes.ANTHROPIC,  # Sets the model service type to Anthropic.
        searchType=SearchTypes.VECTOR,  # Sets the search type to vector.
        anthropic=AnthropicModelPropertiesInput(
            model=model_version,  # Specifies the Anthropic model version to be used.
            temperature=temperature,  # Sets the temperature for model output, affecting randomness.
            probability=probability,  # Sets the probability threshold for certain responses.
            completionTokenLimit=token_limit,  # Limits the number of tokens in the completion output.
        )
    )

    try:
        # Attempt to create the specification using the graphlit client.
        response = await graphlit.client.create_specification(input)

        # Extract and return the specification ID from the response.
        spec_id = response.create_specification.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return spec_id

# Function to create a new conversation using a specification ID and return the conversation ID or an error message.
async def create_conversation(graphlit, spec_id):
    input = ConversationInput(
        name="Conversation",  # Name of the conversation.
        specification=EntityReferenceInput(
            id=spec_id  # Reference to the specification ID to be used for this conversation.
        )
    )

    try:
        # Attempt to create the conversation using the graphlit client.
        response = await graphlit.client.create_conversation(input)

        # Extract and return the conversation ID from the response.
        conv_id = response.create_conversation.id
    except GraphQLClientError as e:
        # Return the error message in case of a GraphQL client error.
        return str(e)

    return conv_id

# Function to send a prompt to an existing conversation and return the generated message and any citations, or an error message.
async def prompt_conversation(graphlit, conv_id, prompt):
    try:
        # Attempt to send a prompt to the conversation using the graphlit client.
        response = await graphlit.client.prompt_conversation(prompt, conv_id)

        # Extract the generated message and citations from the response.
        message = response.prompt_conversation.message.message
        citations = response.prompt_conversation.message.citations

        return message, citations
    except GraphQLClientError as e:
        # Return None for message and the error message in case of a GraphQL client error.
        return None, str(e)

# Helper function to test multiple versions of Anthropic models
async def test_anthropic_models(graphlit, uri, model_versions, prompt):
    # Create a new feed using the URI
    feed_id = await create_feed(graphlit, uri)
    print(f"Feed created: {feed_id}")

    # Iterate over the model versions to create specifications and conversations
        # Iterate over the model versions to create specifications and conversations
    for model_version in model_versions:
        print(f"\nTesting model: {model_version}")

        # Time tracking for model-specific tasks
        model_start_time = time.time()

        # Create a specification for the model version
        spec_id = await create_specification(graphlit, model_version)
        print(f"Specification created: {spec_id}")

        # Create a conversation using the created specification
        conv_id = await create_conversation(graphlit, spec_id)
        print(f"Conversation created: {conv_id}")

        # Send the prompt to the conversation and get the response
        response, error = await prompt_conversation(graphlit, conv_id, prompt)

        # Time tracking for model-specific tasks completed
        model_end_time = time.time()

        # Output response or error
        if response:
            print(f"Response from {model_version}: {response[:]}...")  # Print partial response for brevity
        else:
            print(f"Error with {model_version}: {error}")

        # Print time taken for this model version
        print(f"Time taken for {model_version}: {model_end_time - model_start_time:.2f} seconds")


In [ ]:
# Example usage to test multiple Anthropic model versions
uri = 'https://en.wikipedia.org/wiki/The_Lord_of_the_Rings'
model_versions = [AnthropicModels.CLAUDE_3_HAIKU, AnthropicModels.CLAUDE_3_5_SONNET, AnthropicModels.CLAUDE_3_OPUS]  # List of model versions
prompt = "Summarize the webpage content."

In [ ]:
# Run the test
await test_anthropic_models(graphlit, uri, model_versions, prompt)

Feed created: Creating feed would exceed project feeds quota [3].  Please delete one or more existing feeds, or upgrade to a higher (paid) tier.

Testing model: CLAUDE_3_HAIKU
Specification created: b439a295-0be5-441d-9852-881e4014c328
Conversation created: ee20992d-00ac-4e4a-b861-d07caa32a8cb
Response from CLAUDE_3_HAIKU: The Lord of the Rings is a high fantasy novel written by J.R.R. Tolkien, published in three volumes from 1954-1955. It is set in the fictional world of Middle-earth and follows the journey of the hobbit Frodo Baggins as he and a group of companions embark on a quest to destroy an evil ring of power and defeat the dark lord Sauron. The novel has had a significant impact on popular culture and has been adapted into various forms of media, including films, television series, and video games.

The Lord of the Rings has been translated into over 38 languages and has been widely praised for its imaginative storytelling, complex world-building, and themes of good versus evi

CLAUDE_3_HAIKU:
* Fastest model (5.84 seconds), providing a quick response with high efficiency. Focused primaraly on the overall cultural impact of The Lord of the Rings, highlighting its influence on the fantasy genre, adaptations, and critical acclaim. Not detailing any specifics about the plot which is highly detailed int he wikipedia page. In essence it only summarzd a portion of the page.

CLAUDE_3_5_SONNET:
* Moderate speed (7.10 seconds), returned a balance between speed and detail It Emphasized plot details, particularly Frodo’s quest to destroy the One Ring, while also covering core themes like good vs. evil, friendship, as well as the novel's influence on fantasy literature.

CLAUDE_3_OPUS:
* Slowest model (21.09 seconds), but returned a much more detailed and comprehensive summary. Covering both the plot and Tolkien's inspirations, it took the time to highlight Sauron specifically, the One Ring, and the journey of the hobbits, along with references to mythology and literary influences.